In [1]:
#collapse-hide
import sys
import pyodbc 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from collections import Counter
from sklearn.decomposition import PCA
from adjustText import adjust_text
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')
plt.rcParams["font.family"] = "sans-serif"
PLOTS_DIR = 'images'
%matplotlib inline

In [2]:
#collapse-hide

periodid = 151
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=HUM1006903\SQLEXPRESS;'
                      'Database=oda_20201103;'
                      #'Database=oda;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

sql_query = pd.read_sql_query('SELECT \
        oda.dbo.Afstemning.id AS afstemning, \
        oda.dbo.Sag.periodeid, \
        oda.dbo.Periode.titel AS periode, \
        oda.dbo.Sag.titel AS titel, \
        oda.dbo.Sag.resume, \
        oda.dbo.Afstemning.konklusion, \
        oda.dbo.Sagstrin.dato, \
        oda.dbo.Stemme.typeid AS stemme, \
        oda.dbo.Aktør.fornavn, \
        oda.dbo.Aktør.efternavn, \
        oda.dbo.Aktør.biografi \
    FROM oda.dbo.Afstemning \
    JOIN oda.dbo.Stemme ON oda.dbo.Afstemning.id = oda.dbo.Stemme.afstemningid \
    JOIN oda.dbo.Aktør ON oda.dbo.Aktør.id = oda.dbo.Stemme.aktørid \
    JOIN oda.dbo.Sagstrin ON oda.dbo.Sagstrin.id = oda.dbo.Afstemning.sagstrinid \
    JOIN oda.dbo.Sag ON oda.dbo.Sag.id = oda.dbo.Sagstrin.sagid \
    JOIN oda.dbo.Periode ON oda.dbo.Sag.periodeid = oda.dbo.Periode.id \
    WHERE oda.dbo.Sag.periodeid='+str(periodid)+';', conn)

sql_query.head()

,afstemning,periodeid,periode,titel,resume,konklusion,dato,stemme,fornavn,efternavn,biografi
0,7165,151,2019-20,Forslag til folketingsbeslutning om danske mil...,Med folketingsbeslutningen får regeringen Folk...,"Forslaget blev vedtaget. For stemte 106 (S, V,...",2019-10-24,1,Erling,Bonnesen,<member><url>/medlemmer/mf/e/erling-bonnesen</...
1,7165,151,2019-20,Forslag til folketingsbeslutning om danske mil...,Med folketingsbeslutningen får regeringen Folk...,"Forslaget blev vedtaget. For stemte 106 (S, V,...",2019-10-24,1,Morten,Bødskov,<member><url>/medlemmer/mf/m/morten-boedskov</...
2,7165,151,2019-20,Forslag til folketingsbeslutning om danske mil...,Med folketingsbeslutningen får regeringen Folk...,"Forslaget blev vedtaget. For stemte 106 (S, V,...",2019-10-24,1,Bent,Bøgsted,<member><url>/medlemmer/mf/b/bent-boegsted</ur...
3,7165,151,2019-20,Forslag til folketingsbeslutning om danske mil...,Med folketingsbeslutningen får regeringen Folk...,"Forslaget blev vedtaget. For stemte 106 (S, V,...",2019-10-24,1,Lennart,Damsbo-Andersen,<member><url>/medlemmer/mf/l/lennart-damsbo-an...
4,7165,151,2019-20,Forslag til folketingsbeslutning om danske mil...,Med folketingsbeslutningen får regeringen Folk...,"Forslaget blev vedtaget. For stemte 106 (S, V,...",2019-10-24,1,Louise Schack,Elholm,<member><url>/medlemmer/mf/l/louise-schack-elh...


In [3]:
#collapse-hide
party = []
for bio_string in sql_query['biografi'].values:
    try:
        root = ET.fromstring(bio_string)
        for child in root.findall("./party"):
            party.append(child.text)
    except Exception as e:
        party.append(None)
        continue
sql_query['party'] = party
#sql_query.party.unique()

In [4]:
#collapse-hide

sql_query['navn'] = sql_query[['fornavn', 'efternavn']].agg(' '.join, axis=1)

In [5]:
#collapse-hide

df = sql_query[['afstemning', 'titel', 'resume', 'konklusion', 'navn', 'party', 'stemme']]
df['stemme'].replace(to_replace=2, value=-1, inplace=True)
df['stemme'].replace(to_replace=4, value=0, inplace=True)
df.tail()

,afstemning,titel,resume,konklusion,navn,party,stemme
57812,7490,"Folketinget konstaterer, at det aftalte loft o...",,"Forslaget blev vedtaget. For stemte 54 (S, RV,...",Birgitte Bergman,Det Konservative Folkeparti,0
57813,7490,"Folketinget konstaterer, at det aftalte loft o...",,"Forslaget blev vedtaget. For stemte 54 (S, RV,...",Birgitte Vind,Socialdemokratiet,0
57814,7490,"Folketinget konstaterer, at det aftalte loft o...",,"Forslaget blev vedtaget. For stemte 54 (S, RV,...",Pernille Vermund,Nye Borgerlige,0
57815,7490,"Folketinget konstaterer, at det aftalte loft o...",,"Forslaget blev vedtaget. For stemte 54 (S, RV,...",Theresa Berg Andersen,Socialistisk Folkeparti,0
57816,7490,"Folketinget konstaterer, at det aftalte loft o...",,"Forslaget blev vedtaget. For stemte 54 (S, RV,...",Ruben Kidde,Radikale Venstre,0


In [6]:
#show
Counter(df.stemme)

Counter({1: 22567, -1: 11244, 3: 20853, 0: 3153})

In [22]:
#collapse-hide
#Vi kan også prøve at kigge på løsgængerne stemmeadfærd:
s = []
for lg in df.navn.unique():
    #print(lg, Counter(sql_query[sql_query.navn == lg].stemme)[3])
    s.append((lg, Counter(sql_query[sql_query.navn == lg].stemme)[3]))
sorted_fravaer = sorted(s, reverse=True, key=lambda tup: tup[1])
sorted_fravaer
#som viser at Uffe Elbæk og Simon Emil Ammitzbøll-Bille (begge i midten af x-aksen) været fraværende ved hhv 71% og 84% af alle registrerede afstemninger i folketingssalen (da der er møde- og stemmepligt, og de to næppe har lavet clearingsaftaler, er det måske en sag der er værd at undersøge nærmere). Sikandar Siddique (på venstre side af x-aksen tæt på SF) derimod deltager flittigt i afstemningerne.

[('Mette Frederiksen', 283),
 ('Claus Hjort Frederiksen', 280),
 ('Edmund Joensen', 280),
 ('Sjúrður Skaale', 278),
 ('Aaja Chemnitz Larsen', 278),
 ('Morten Østergaard', 276),
 ('Simon Emil Ammitzbøll-Bille', 272),
 ('Kristian Jensen', 262),
 ('Pia Kjærsgaard', 261),
 ('Pia Olsen Dyhr', 252),
 ('Lars Løkke Rasmussen', 246),
 ('Dan Jørgensen', 237),
 ('Hans Christian Schmidt', 235),
 ('Nicolai Wammen', 232),
 ('Magnus Heunicke', 230),
 ('Aki-Matilda Høegh-Dam', 230),
 ('Uffe Elbæk', 229),
 ('Pernille Rosenkrantz-Theil', 228),
 ('Jakob Ellemann-Jensen', 225),
 ('Henrik Dam Kristensen', 216),
 ('Mogens Jensen', 206),
 ('Jacob Jensen', 204),
 ('Ane Halsboe-Jørgensen', 204),
 ('Astrid Krag', 202),
 ('Søren Søndergaard', 201),
 ('Martin Lidegaard', 199),
 ('Pernille Skipper', 199),
 ('Jacob Mark', 197),
 ('Rasmus Prehn', 190),
 ('Ina Strøjer-Schmidt', 186),
 ('Eva Kjer Hansen', 184),
 ('Marcus Knuth', 183),
 ('Jens Rohde', 182),
 ('Brigitte Klintskov Jerkel', 180),
 ('Søren Espersen', 179),